In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from FoodSeg103_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the images as batches

In [2]:
img_nums = [0]#[n for n in range(0, 4900, 500)] #2750 #1234 #2314
imgs, gts = load_food_batch(img_nums)
num_imgs = len(imgs)
print(f"Total images: {num_imgs}")

Total images: 1


Check and filter for resolution

In [ ]:
resol = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}

# Check for images with resolution over a certain threshold
thresh = 200_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

In [ ]:
# Filter out those images
imgs = {im_num: im for im_num, im in imgs.items() if resol[im_num] < thresh}
num_imgs = len(imgs)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Define parameters

In [3]:
# Which scribbles to use
mode = "all"
bins = [0.05, 0.3, 1.5]
scribble_width = 3
# suff = ["TEST"] #standing for width 3
s = ""

# Which prediction models to use
convpaint_layers = [[0], [0,2]]
convpaint_scalings = [[1], [1,2]]#, [1,2,4], [1,2,4,8]]
use_dino = True
use_ilastik = True

time_seed = 123

# Where to find and save the data
folder_path = "./FoodSeg103_results/data/food_run03b"

Loop and analyse

In [4]:
df_list = []

for img_num, img in imgs.items():
    ground_truth = gts[img_num]
    for bin in bins:
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        for layers in convpaint_layers:
            for scalings in convpaint_scalings:
                print(f"ConvPaint: layers {layers}, scalings {scalings}")
                pred_tag = generate_convpaint_tag(layers, scalings)
                # res = analyse_food_single_file(ground_truth, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag=pred_tag, show_res=False, image=None)
                res = time_food(img, ground_truth, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag=pred_tag, layer_list=layers, scalings=scalings, random_state=time_seed)
                df_list.append(res)
        if use_ilastik:
            print("Ilastik")
            # res = analyse_food_single_file(ground_truth, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="ilastik", show_res=False, image=None)
            res = time_food(img, ground_truth, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="ilastik", random_state=time_seed)
            df_list.append(res)
        if use_dino:
            print("DINO")
            # res = analyse_food_single_file(ground_truth, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="dino", show_res=False, image=None)
            res = time_food(img, ground_truth, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="dino", random_state=time_seed)
            df_list.append(res)
        print("\n")

df = pd.concat(df_list, ignore_index=True)
df[:5]

IMG 0: bin 0.05, suff 
ConvPaint: layers [0], scalings [1]
ConvPaint: layers [0], scalings [1, 2]
ConvPaint: layers [0, 2], scalings [1]
ConvPaint: layers [0, 2], scalings [1, 2]
Ilastik
DINO


C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")




IMG 0: bin 0.3, suff 
ConvPaint: layers [0], scalings [1]
ConvPaint: layers [0], scalings [1, 2]
ConvPaint: layers [0, 2], scalings [1]
ConvPaint: layers [0, 2], scalings [1, 2]
Ilastik
DINO


IMG 0: bin 1.5, suff 
ConvPaint: layers [0], scalings [1]
ConvPaint: layers [0], scalings [1, 2]
ConvPaint: layers [0, 2], scalings [1]
ConvPaint: layers [0, 2], scalings [1, 2]
Ilastik
DINO




C:\Users\roman\AppData\Local\Temp\ipykernel_19272\2702792049.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_list, ignore_index=True)


,img_num,prediction tag,prediction type,layers,scalings,scribbles mode,scribbles bin,suffix,min_class_pix_gt,max_class_pix_gt,...,max_class_pix_labelled,pix_in_img,perc. labelled,t_load,t_features_full,t_features_train,t_train,t_pred,t_selfpred,scribbles
0,0,convpaint_l-0_s-1,convpaint,0,1,all,0.05,,29097,73227,...,137,196608,0.177511,1.667416,0.152833,0.401976,0.791721,1.487465,2.681162,./FoodSeg103_results/data/food_run03b/0000_scr...
1,0,convpaint_l-0_s-1-2,convpaint,0,1-2,all,0.05,,29097,73227,...,137,196608,0.177511,1.896743,0.164187,0.765625,0.602351,1.632151,3.000127,./FoodSeg103_results/data/food_run03b/0000_scr...
2,0,convpaint_l-0-2_s-1,convpaint,0-2,1,all,0.05,,29097,73227,...,137,196608,0.177511,1.977408,0.535775,1.100428,0.580704,1.913709,3.594841,./FoodSeg103_results/data/food_run03b/0000_scr...
3,0,convpaint_l-0-2_s-1-2,convpaint,0-2,1-2,all,0.05,,29097,73227,...,137,196608,0.177511,1.695210,0.749762,1.983390,0.600027,2.482972,5.066390,./FoodSeg103_results/data/food_run03b/0000_scr...
4,0,ilastik,classical filters,None,None,all,0.05,,29097,73227,...,137,196608,0.177511,NaN,0.801113,0.801113,0.351513,0.865535,2.018162,./FoodSeg103_results/data/food_run03b/0000_scr...


In [10]:
# df.groupby(['prediction type'])[['accuracy']].mean()
df.groupby(['prediction tag'])[['t_load', 't_features_full', 't_features_train', 't_train', 't_pred', 't_selfpred']].mean()

,t_load,t_features_full,t_features_train,t_train,t_pred,t_selfpred
prediction tag,,,,,,
convpaint_l-0-2_s-1,2.076571,0.585003,1.262774,14.057445,2.702745,18.022964
convpaint_l-0-2_s-1-2,1.858266,0.801969,2.203666,14.953279,3.397979,20.554923
convpaint_l-0_s-1,1.772618,0.109342,0.373488,9.185579,1.937468,11.496535
convpaint_l-0_s-1-2,2.042405,0.166361,0.832156,14.170528,2.182584,17.185267
dino,0.506566,1.434776,1.434776,1.802519,0.016294,3.253589
ilastik,NaN,0.831894,0.831894,6.003223,1.143374,7.978491


Save the data frame in a csv file:

In [ ]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
# file_name = f"FoodSeg103_{time_stamp}.csv"
file_name = f"FoodSeg103_times_{time_stamp}.csv"
df.to_csv(file_name, index=False)